Load requiered modules

In [78]:
import openvino as ov
from torchvision import transforms
import torch
from PIL import Image 
import numpy as np
from numpy.linalg import norm
from utils import TrainManager
import torch.nn.functional as F

DEVICE = 'CPU'
MODEL_PATH = 'output_models_full/efficientnet_v2_s_512/epoch=4-EER_VER=8.65.ckpt'
BACKBONE = 'efficientnet_v2_s'
NUM_IDENTITIES = 11670
NUM_FEATURE = 512
INPUT_SIZE = 324

transform = transforms.Compose([
                    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

Create Openvino core, load the model and compile the latter

In [64]:
# Create OpenVINO Core object instance
core = ov.Core()

model = TrainManager.load_from_checkpoint(checkpoint_path=MODEL_PATH, model=BACKBONE, num_identities=NUM_IDENTITIES, num_features=NUM_FEATURE)
model.eval()
model.to('cpu')

# Convert model to openvino.runtime.Model object
ov_model = ov.convert_model(model.backbone, input=[[1,3,INPUT_SIZE,INPUT_SIZE]])

compiled_model = core.compile_model(ov_model, device_name=DEVICE)

Load the input image and transform

In [65]:
#loading input images for subject #1 and subject #2
palm_s1 = Image.open('examples/subject1/palm/83d72e4f-ea43-417b-80a1-a3e7ba61b096.jpg')
peace_s1 = Image.open('examples/subject1/peace/5105a824-0c49-41d9-b21b-0d3301918eb9.jpg')

palm_s2 = Image.open('examples/subject2/palm/642321dd-5101-42f5-b974-4b9236f85fc7.jpg')
peace_s2 = Image.open('examples/subject2/peace/231de2d7-f406-44fd-8c17-a3c8854da40b.jpg')

#Transforming the images for both subjects
palm_s1 = torch.unsqueeze(transform(palm_s1), dim=0)
peace_s1 = torch.unsqueeze(transform(peace_s1), dim=0) 

palm_s2 = torch.unsqueeze(transform(palm_s2), dim=0)
peace_s2 = torch.unsqueeze(transform(peace_s2), dim=0) 

palm_s1.shape

torch.Size([1, 3, 324, 324])

In [66]:
# Run model inference for each subject's image and get the feature embedding
palm_s1_emb = compiled_model(palm_s1)[0][0]
palm_s1_emb = palm_s1_emb / np.linalg.norm(palm_s1_emb)

peace_s1_emb = compiled_model(peace_s1)[0][0]
peace_s1_emb = peace_s1_emb / np.linalg.norm(peace_s1_emb)

palm_s2_emb = compiled_model(palm_s2)[0][0]
palm_s2_emb = palm_s2_emb / np.linalg.norm(palm_s2_emb)

peace_s2_emb = compiled_model(peace_s2)[0][0]
peace_s2_emb = peace_s2_emb / np.linalg.norm(peace_s2_emb)

Check the time spent to compute an embedding using static onnx model

In [117]:
%%timeit
palm_s1_emb = compiled_model(palm_s1)[0]

38.3 ms ± 625 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Check the time spent to compute an embedding using the pytorch model

In [111]:
%%timeit
palm_s1_emb_pytorch = model.backbone(palm_s1)

110 ms ± 3.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Check whether pytorch model and the openvino produce the same embedding
 

In [116]:
palm_s1_emb_pytorch_np = F.normalize(palm_s1_emb_pytorch)
palm_s1_emb_pytorch_np = palm_s1_emb_pytorch.detach().cpu().numpy()
palm_s1_emb_pytorch_np = palm_s1_emb_pytorch_np[0]

value_s1 = np.dot(palm_s1_emb,palm_s1_emb_pytorch_np)/(norm(palm_s1_emb)*norm(palm_s1_emb_pytorch_np))

print('Similarity between the same embeddings -> {}'.format(value_s1))

Similarity between the same embeddings -> 1.0000001192092896


Compute similarity between samples from the same subjects

In [118]:
value_s1 = np.dot(palm_s1_emb,peace_s1_emb)/(norm(palm_s1_emb)*norm(peace_s1_emb))
value_s2 = np.dot(palm_s2_emb,peace_s2_emb)/(norm(palm_s1_emb)*norm(peace_s2_emb))

print('Similarity between samples of subject 1 -> {}'.format(value_s1))
print('Similarity between samples of subject 2 -> {}'.format(value_s2))

Similarity between samples of subject 1 -> 0.9951640367507935
Similarity between samples of subject 2 -> 0.9912264347076416


Compute similarity between samples from the different subjects

In [119]:
value_palm_s1_2 = np.dot(palm_s1_emb,palm_s2_emb)/(norm(palm_s1_emb)*norm(palm_s2_emb))
value_peace_s1_2 = np.dot(peace_s1_emb,peace_s2_emb)/(norm(peace_s1_emb)*norm(peace_s2_emb))

print('Similarity between palm samples of different subjects -> {}'.format(value_palm_s1_2))
print('Similarity between peace samples of different subjects -> {}'.format(value_peace_s1_2))

Similarity between palm samples of different subjects -> 0.9408684372901917
Similarity between peace samples of different subjects -> 0.94172602891922
